## Basic Structured Operations


### Step 1: Initialize PySpark Session


In [2]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder.appName("day2").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/31 09:28:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Step 2: Load the Dataset


In [3]:
# Load the Chipotle dataset into a Spark DataFrame
data_path = "../data/occupation.csv"  # Replace with the actual path
occupation = spark.read.csv(data_path, header=True, inferSchema=True)


In [4]:
occupation.printSchema()

root
 |-- user_id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- zip_code: string (nullable = true)



### Problem 1: Selecting Specific Columns
Problem: Select the "user_id," "age," and "occupation" columns from the occupation DataFrame.

In [5]:
#To select required columns from occupation
selected_columns= occupation.select("user_id", "age", "occupation")

#to show the seleted columns
selected_columns.show()


+-------+---+-------------+
|user_id|age|   occupation|
+-------+---+-------------+
|      1| 24|   technician|
|      2| 53|        other|
|      3| 23|       writer|
|      4| 24|   technician|
|      5| 33|        other|
|      6| 42|    executive|
|      7| 57|administrator|
|      8| 36|administrator|
|      9| 29|      student|
|     10| 53|       lawyer|
|     11| 39|        other|
|     12| 28|        other|
|     13| 47|     educator|
|     14| 45|    scientist|
|     15| 49|     educator|
|     16| 21|entertainment|
|     17| 30|   programmer|
|     18| 35|        other|
|     19| 40|    librarian|
|     20| 42|    homemaker|
+-------+---+-------------+
only showing top 20 rows



### Problem 2: Filtering Rows based on Condition
Problem: Find the users who are older than 30 years from the occupation DataFrame.

In [6]:
occupation.select("*").where("age > 30").show()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|      2| 53|     F|        other|   94043|
|      5| 33|     F|        other|   15213|
|      6| 42|     M|    executive|   98101|
|      7| 57|     M|administrator|   91344|
|      8| 36|     M|administrator|   05201|
|     10| 53|     M|       lawyer|   90703|
|     11| 39|     F|        other|   30329|
|     13| 47|     M|     educator|   29206|
|     14| 45|     M|    scientist|   55106|
|     15| 49|     F|     educator|   97301|
|     18| 35|     F|        other|   37212|
|     19| 40|     M|    librarian|   02138|
|     20| 42|     F|    homemaker|   95660|
|     25| 39|     M|     engineer|   55107|
|     26| 49|     M|     engineer|   21044|
|     27| 40|     F|    librarian|   30030|
|     28| 32|     M|       writer|   55369|
|     29| 41|     M|   programmer|   94043|
|     34| 38|     F|administrator|   42141|
|     39| 41|     M|entertainmen

### Problem 3: Counting and Grouping
Problem: Count the number of users in each occupation from the occupation DataFrame.

In [7]:
occupation.groupBy("occupation").count().show()

+-------------+-----+
|   occupation|count|
+-------------+-----+
|    librarian|   51|
|      retired|   14|
|       lawyer|   12|
|         none|    9|
|       writer|   45|
|   programmer|   66|
|    marketing|   26|
|        other|  105|
|    executive|   32|
|    scientist|   31|
|      student|  196|
|     salesman|   12|
|       artist|   28|
|   technician|   27|
|administrator|   79|
|     engineer|   67|
|   healthcare|   16|
|     educator|   95|
|entertainment|   18|
|    homemaker|    7|
+-------------+-----+
only showing top 20 rows



### Problem 4: Adding a New Column
Problem: Add a new column "age_group" to the occupation DataFrame based on the age of the users. Divide users into age groups: "18-25", "26-35", "36-50", and "51+".

In [8]:
from pyspark.sql.functions import when, lit

occupation.withColumn("age_group", \
   when((occupation.age >= 18) & (occupation.age <= 25), lit("18-25")) \
     .when((occupation.age >= 26) & (occupation.age <= 35), lit("26-35")) \
      .when((occupation.age >= 36) & (occupation.age <= 50), lit("36-50")) \
     .otherwise(lit("51+")) \
  ).show()

+-------+---+------+-------------+--------+---------+
|user_id|age|gender|   occupation|zip_code|age_group|
+-------+---+------+-------------+--------+---------+
|      1| 24|     M|   technician|   85711|    18-25|
|      2| 53|     F|        other|   94043|      51+|
|      3| 23|     M|       writer|   32067|    18-25|
|      4| 24|     M|   technician|   43537|    18-25|
|      5| 33|     F|        other|   15213|    26-35|
|      6| 42|     M|    executive|   98101|    36-50|
|      7| 57|     M|administrator|   91344|      51+|
|      8| 36|     M|administrator|   05201|    36-50|
|      9| 29|     M|      student|   01002|    26-35|
|     10| 53|     M|       lawyer|   90703|      51+|
|     11| 39|     F|        other|   30329|    36-50|
|     12| 28|     F|        other|   06405|    26-35|
|     13| 47|     M|     educator|   29206|    36-50|
|     14| 45|     M|    scientist|   55106|    36-50|
|     15| 49|     F|     educator|   97301|    36-50|
|     16| 21|     M|entertai

### Problem 5: Creating DataFrames and Converting to Spark Types
Problem: Given the provided code snippet, create a DataFrame df using the given data and schema. The schema includes columns for firstname, middlename, lastname, id, gender, and salary. After creating the DataFrame, print its schema and display its content without truncation.

In [9]:
from pyspark.sql.types import *

schema = StructType([
    StructField("firstname", StringType(), True),
    StructField("middlename", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("id", IntegerType(), True),
    StructField("gender", StringType(), True),
    StructField("salary", DoubleType(), True)
])

data = [("James", None, "Smith", 36636, "M", 3000.0),
        ("Michael", "Rose", None, 40288, "M", 4000.0),
        ("Robert", None, "Williams", 42114, "M", 4000.0),
        ("Maria", "Anne", "Jones", 39192, "F", 4000.0),
        ("Jen", "Mary", "Brown", None, "F", -1.0)]

myDf= spark.createDataFrame(data, schema)
myDf.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: double (nullable = true)



### Problem 6: Adding and Renaming Columns
Problem: Add a new column "gender" to the existing DataFrame and rename the "Age" column to "Years".

In [10]:
new_occupation= occupation.withColumn("gender", lit("Unknown")).withColumnRenamed("age","Years")
new_occupation.show()



+-------+-----+-------+-------------+--------+
|user_id|Years| gender|   occupation|zip_code|
+-------+-----+-------+-------------+--------+
|      1|   24|Unknown|   technician|   85711|
|      2|   53|Unknown|        other|   94043|
|      3|   23|Unknown|       writer|   32067|
|      4|   24|Unknown|   technician|   43537|
|      5|   33|Unknown|        other|   15213|
|      6|   42|Unknown|    executive|   98101|
|      7|   57|Unknown|administrator|   91344|
|      8|   36|Unknown|administrator|   05201|
|      9|   29|Unknown|      student|   01002|
|     10|   53|Unknown|       lawyer|   90703|
|     11|   39|Unknown|        other|   30329|
|     12|   28|Unknown|        other|   06405|
|     13|   47|Unknown|     educator|   29206|
|     14|   45|Unknown|    scientist|   55106|
|     15|   49|Unknown|     educator|   97301|
|     16|   21|Unknown|entertainment|   10309|
|     17|   30|Unknown|   programmer|   06355|
|     18|   35|Unknown|        other|   37212|
|     19|   4

### Problem 7: Filtering Rows and Sorting
Problem: Filter out users who are younger than 30 years and sort the DataFrame by age in descending order.

In [11]:
from pyspark.sql.functions import col, desc

filtered_occupation= occupation.filter(col("age")>30)
# sorted_occupation = filtered_occupation.sort(desc("age"))

sorted_occupation = filtered_occupation.orderBy(col("age").desc())

sorted_occupation.show()

+-------+---+------+-------------+--------+
|user_id|age|gender|   occupation|zip_code|
+-------+---+------+-------------+--------+
|    481| 73|     M|      retired|   37771|
|    767| 70|     M|     engineer|   00000|
|    803| 70|     M|administrator|   78212|
|    860| 70|     F|      retired|   48322|
|    559| 69|     M|    executive|   10022|
|    585| 69|     M|    librarian|   98501|
|    349| 68|     M|      retired|   61455|
|    573| 68|     M|      retired|   48911|
|    211| 66|     M|     salesman|   32605|
|    318| 65|     M|      retired|   06518|
|    564| 65|     M|      retired|   94591|
|    651| 65|     M|      retired|   02903|
|    423| 64|     M|        other|   91606|
|    845| 64|     M|       doctor|   97405|
|    364| 63|     M|     engineer|   01810|
|    777| 63|     M|   programmer|   01810|
|    858| 63|     M|     educator|   09645|
|    266| 62|     F|administrator|   78756|
|    520| 62|     M|   healthcare|   12603|
|    106| 61|     M|      retire

### Problem 8: Repartitioning and Collecting Rows
Problem: Repartition the DataFrame into 2 partitions without shuffling the data, then collect and display all rows in the driver and print number of partitions

In [12]:

# Coalesce the DataFrame into 2 partitions without shuffling
myDf = myDf.coalesce(2)

# Collect all rows to the driver and display them
rows = myDf.collect()
for row in rows:
    print(row)




Row(firstname='James', middlename=None, lastname='Smith', id=36636, gender='M', salary=3000.0)
Row(firstname='Michael', middlename='Rose', lastname=None, id=40288, gender='M', salary=4000.0)
Row(firstname='Robert', middlename=None, lastname='Williams', id=42114, gender='M', salary=4000.0)
Row(firstname='Maria', middlename='Anne', lastname='Jones', id=39192, gender='F', salary=4000.0)
Row(firstname='Jen', middlename='Mary', lastname='Brown', id=None, gender='F', salary=-1.0)


In [13]:
# Check the current number of partitions
print("Number of partitions a:", myDf.rdd.getNumPartitions())

Number of partitions a: 2


### Additional questions:

Use both spark SQL and Pyspark to obtain answer wherever relevant

#### Filter out rows where the age is greater than 30 and create a new DataFrame. Then, add a new column named "is_elderly" with a value of "True" for these rows and "False" otherwise.Rename the "gender" column to "sex".

In [14]:
# Spark SQL
occupation.createOrReplaceTempView("people")
query="""SELECT *,\
           CASE WHEN age > 30 THEN True ELSE False END as is_elderly,\
           gender as sex\
    FROM people\
    WHERE age > 30"""
spark.sql(query).show()

+-------+---+------+-------------+--------+----------+---+
|user_id|age|gender|   occupation|zip_code|is_elderly|sex|
+-------+---+------+-------------+--------+----------+---+
|      2| 53|     F|        other|   94043|      true|  F|
|      5| 33|     F|        other|   15213|      true|  F|
|      6| 42|     M|    executive|   98101|      true|  M|
|      7| 57|     M|administrator|   91344|      true|  M|
|      8| 36|     M|administrator|   05201|      true|  M|
|     10| 53|     M|       lawyer|   90703|      true|  M|
|     11| 39|     F|        other|   30329|      true|  F|
|     13| 47|     M|     educator|   29206|      true|  M|
|     14| 45|     M|    scientist|   55106|      true|  M|
|     15| 49|     F|     educator|   97301|      true|  F|
|     18| 35|     F|        other|   37212|      true|  F|
|     19| 40|     M|    librarian|   02138|      true|  M|
|     20| 42|     F|    homemaker|   95660|      true|  F|
|     25| 39|     M|     engineer|   55107|      true|  

In [15]:
# Pyspark

# Filter out rows where age is greater than 30
filtered_occupation = occupation.filter(occupation["age"] > 30)

# Add a new column "is_elderly" based on age
result_occupation = filtered_occupation.withColumn("is_elderly", 
when(occupation.age > 30, "True").otherwise("False"))

# Rename the "gender" column to "sex"
result_occupation = result_occupation.withColumnRenamed("gender", "sex")

result_occupation.show()

+-------+---+---+-------------+--------+----------+
|user_id|age|sex|   occupation|zip_code|is_elderly|
+-------+---+---+-------------+--------+----------+
|      2| 53|  F|        other|   94043|      True|
|      5| 33|  F|        other|   15213|      True|
|      6| 42|  M|    executive|   98101|      True|
|      7| 57|  M|administrator|   91344|      True|
|      8| 36|  M|administrator|   05201|      True|
|     10| 53|  M|       lawyer|   90703|      True|
|     11| 39|  F|        other|   30329|      True|
|     13| 47|  M|     educator|   29206|      True|
|     14| 45|  M|    scientist|   55106|      True|
|     15| 49|  F|     educator|   97301|      True|
|     18| 35|  F|        other|   37212|      True|
|     19| 40|  M|    librarian|   02138|      True|
|     20| 42|  F|    homemaker|   95660|      True|
|     25| 39|  M|     engineer|   55107|      True|
|     26| 49|  M|     engineer|   21044|      True|
|     27| 40|  F|    librarian|   30030|      True|
|     28| 32

#### Calculate the average age of male and female users separately. Present the result in a new DataFrame with columns "gender" and "avg_age".

In [16]:
# Spark SQL



In [17]:
# Pyspark



#### Add a new column named "full_name" to the dataset by concatenating the "user_id" and "occupation" columns. Then, rename the "zip_code" column to "postal_code" in the same DataFrame.

In [18]:
# Spark SQL



In [19]:
# Pyspark



#### Filter out rows where occupation is 'technician', select only the "user_id" and "age" columns, and then add a new column "age_diff" that calculates the difference between the user's age and the average age in the dataset.

In [20]:
# Spark SQL



In [21]:
# Pyspark



#### Divide the dataset into two DataFrames: one with male users and another with female users. Repartition both DataFrames to have 2 partitions each. Then, union these two DataFrames together and display the resulting DataFrame.

In [22]:
male_user = occupation.filter(occupation["gender"] == "M")
female_user = occupation.filter(occupation["gender"] == "F")

male_user= male_user.repartition(2)
female_user= female_user.repartition(2)

male_user.show(2)
female_user.show(2)

all_users = male_user.union(female_user)
all_users.show(5)

male_user.count()
female_user.count()
all_users.count()


+-------+---+------+----------+--------+
|user_id|age|gender|occupation|zip_code|
+-------+---+------+----------+--------+
|    896| 28|     M|    writer|   91505|
|    156| 25|     M|  educator|   08360|
+-------+---+------+----------+--------+
only showing top 2 rows

+-------+---+------+----------+--------+
|user_id|age|gender|occupation|zip_code|
+-------+---+------+----------+--------+
|    505| 27|     F|     other|   20657|
|    241| 26|     F|   student|   20001|
+-------+---+------+----------+--------+
only showing top 2 rows

+-------+---+------+----------+--------+
|user_id|age|gender|occupation|zip_code|
+-------+---+------+----------+--------+
|    896| 28|     M|    writer|   91505|
|    156| 25|     M|  educator|   08360|
|    568| 39|     M|  educator|   01915|
|    624| 19|     M|   student|   30067|
|    832| 24|     M|technician|   77042|
+-------+---+------+----------+--------+
only showing top 5 rows



943

#### Create and fill a new DataFrame named user_ratings with columns user_id and rating max 10 column. Both user_data and user_ratings share the user_id column. Combine these two DataFrames to create a new DataFrame that includes user information and their corresponding ratings. Make sure to keep only the users present in both DataFrames.